In [2]:
import ast
import contextily as ctx
import psycopg2.extras as pgx
import psycopg2 as pg
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

import sqlalchemy
from sqlalchemy import create_engine
import pyarrow as pa
import pyarrow.parquet as pq
from IPython.display import HTML
import base64

from pylab import *
%matplotlib inline
import geopandas as gpd

In [50]:
# Load data
df = pd.read_csv('../../../data/TTS_LBNL_public_file_21-Aug-2024_all.csv')
cpi = pd.read_csv('../../../data/inflation_multipliers.csv')


/var/folders/sr/hy8nzjhj7xz25lt74m44nv140000gn/T/ipykernel_7608/2160336936.py:2: DtypeWarning: Columns (1,2,3,8,11,15,16,18,20,29,32,35,38,39,40,41,42,43,44,45,46,53,54,56,57,59,60,63,64,65,66,67,68,74,75,79,80) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../../../data/TTS_LBNL_public_file_21-Aug-2024_all.csv')


In [51]:
# Extract year, month, day

# 1) parse into a proper datetime
df['installation_dt'] = pd.to_datetime(df['installation_date'], format='%d-%b-%Y', errors='coerce')

# 2) extract year, month, day
df['year']  = df['installation_dt'].dt.year
df['month'] = df['installation_dt'].dt.month
df['day']   = df['installation_dt'].dt.day

In [66]:
# Extract battery data
df_batt = df.loc[(df['technology_type'].isin(['pv-storage','storage-only'])) & (df['battery_price'] > 0) & (df['battery_rated_capacity_kWh'] > 0)]

# Calculate price per kWh
df_batt['price_per_kwh'] = df_batt['battery_price']/df_batt['battery_rated_capacity_kWh']

# Join to inflation data
df_batt = df_batt.loc[df_batt['year']>= 2020][['year', 'month', 'price_per_kwh']].merge(cpi, on = ['year', 'month']).sort_values(['year', 'month'])

# Adjust for inflation
df_batt['price_per_kwh_adj'] = df_batt['price_per_kwh'] * df_batt['inflation_multiplier']


/var/folders/sr/hy8nzjhj7xz25lt74m44nv140000gn/T/ipykernel_7608/1163411432.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_batt['price_per_kwh'] = df_batt['battery_price']/df_batt['battery_rated_capacity_kWh']


In [67]:
df_batt['price_per_kwh_adj'].describe()

count    12997.000000
mean      1242.202496
std        411.038445
min        110.955814
25%       1078.379946
50%       1178.175340
75%       1308.760506
max      12794.007115
Name: price_per_kwh_adj, dtype: float64

In [63]:
df['battery_price'].unique()

array([-1.000000e+00,  1.500000e+04,  2.360600e+04, ...,  2.566286e+06,
        1.727000e+04,  1.986662e+06])

In [64]:
df['technology_type'].unique()

array(['pv-only', 'pv+storage', 'storage-only', 'redacted', '-1', -1],
      dtype=object)